# Praca domowa z ML numer 11

<a href="https://colab.research.google.com/github/tomczj/ML24_25/blob/main/Autoencoders/homework_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transponowana konwolucja - co to jest?

Transponowana konwolucja to operacja, która (mówiąc niedokładnie) ma „odwrócić” działanie zwykłej konwolucji w sieciach CNN. Precyzyjniej rzecz ujmując, transponowana konwolucja pozwala odzyskać wcześniejsze wymiary przestrzenne danych. Warto tutaj przypomnieć, że zastosowanie warstw konwolucyjnych zazwyczaj zmniejsza rozmiary przestrzenne danych, stąd potrzeba operacji, która może „odwrócić” ten efekt.

Transponowana konwolucja nie jest jednak <strong>przekształceniem odwrotnym</strong> w ścisłym sensie (co zobaczymy), ponieważ jej zastosowanie nie musi (a wręcz zwykle nie pozwala) na odzyskanie dokładnych danych wejściowych.

Warto już teraz zaznaczyć, że transponowana konwolucja działa w pewnym sensie odwrotnie do klasycznej konwolucji — z mniejszej liczby wartości „rozlewa” je na większy obszar, zwiększając rozmiary przestrzenne i wytwarzając więcej danych. Dobrze ilustruje to poniższy obrazek, który pozwoliłem sobie wkleić, ponieważ jasno przedstawia ideę i różnice między konwolucją, dekonwolucją oraz transponowaną konwolucją. Warto zaznaczyć, że dekonwolucja ma na celu dokładne odwrócenie działania konwolucji, natomiast transponowana konwolucja może prowadzić do danych innych niż pierwotne dane wejściowe.

<img src="https://raw.githubusercontent.com/tomczj/ML24_25/main/Autoencoders/cool_diagram.webp">

###### Źródło: https://towardsdatascience.com/what-is-transposed-convolutional-layer-40e5e6e31c11/

# Jaka jest różnica między zwykłą konwolucją, a konwolucją transponowaną?

Najlepiej zrozumieć działanie konwolucji transponowanej, zestawiając ją bezpośrednio z klasyczną konwolucją:

| Cecha        | Splot (Convolution)                           | Splot transponowany (Transposed Convolution)   |
| ------------ | --------------------------------------------- | ---------------------------------------------- |
| Działanie    | Zmniejsza rozmiar przestrzenny (downsampling) | Zwiększa rozmiar przestrzenny (upsampling)     |
| Wejście      | Duża mapa cech                                | Mniejsza (skompresowana) mapa cech             |
| Wyjście      | Mniejsza mapa cech                            | Większa (odtworzona) mapa cech                 |
| Obliczenia   | Przesuwa jądro po wejściu                     | Rozprowadza wejście po wyjściu za pomocą jądra |
| Zastosowanie | Ekstrakcja cech                               | Rekonstrukcja lub segmentacja                  |

Jak wspomniano wcześniej, głównym zastosowaniem konwolucji transponowanych jest rekonstrukcja danych o większej rozdzielczości na podstawie danych o niższej rozdzielczości. Innymi słowy – mając mniejszą mapę cech, możemy dzięki konwolucji transponowanej odpowiednio rozmieścić te wartości w większej przestrzeni, aby uzyskać dane wyjściowe o wyższej rozdzielczości. Jak widać zastosowanie transponowanych konwolucji wręcz się narzuca - na przykład do zwiększenia rozdzielczości zdjęcia, nawet jeśli było ono zapisane wcześniej w słabej jakości.

###### Źródło: https://guipleite.medium.com/an-introduction-to-convolutional-neural-networks-for-image-upscaling-using-sub-pixel-cnns-5d9ea911c557

# Jak transponowane konwolucje przeprowadzają upsampling?

## Najpierw krótko o składowych, czyli co to stride, padding i rozmiar jądra i na co wpływają.

Jak już wspomnieliśmy transponowana konwolucja to coś co ma przypominać przekształcenie odwrotne do konowulcji w CNN. Zamiast jednak odzyskać pełne dane, chcemy odzyskać przede wszystkim wymiar przestrzenny. Najłatwiej będzie to wyjaśnić myśląc o zdjęciach. Załóżmy, że mamy zdjęcie, które zostało zapsiane w słabej rozdzielczości, chcielibyśmy jednak mieć je w większej rozdzielczości, ale jednocześnie zachować jego charakterystykę, to znaczy nie chcemy wstawiać losowych pikseli a te, które najbardziej będą pasować. Przejdźmy teraz do dokładnego opisu. 

Przyjmijmy następujące wielkości:
- `H_in x W_in` - wysokość i szerokość naszego obrazka.
- `K x K` - rozmiar jądra jakim będziemy działać na obrazek. Intuicyjnie będzie nam to mówić jak bardzo rozciągamy każdy piksel. Można o tym myśleć, że z każdego pojedyńczego piksela będziemy robić macierz KxK, gdzie wartość liczbową mnożymy właśnie przez macierz wag (czyli jądro).
- `S` - stride. Warto już na wstępie zaznaczyć, że jest to inny stride, to znaczy odpowiada za coś innego niż stride w przypadku CNN.
- `P` - padding, czyli usunięcie pikseli z brzegów ostatecznego obrazka. Jak zobaczymy nie jest to dokładnie to samo co padding w CNN.

Idea jest teraz bardzo prosta, chcemy odtworzyć obrazek o wyższej rozdzielczości odpowiednio manipulując i replikując wartości (pikselami) z obrazka o mniejszej rozdzielczości. Zanim jednak przejdziemy do dokładnego opisu postępowania krok po kroku to warto się zastanowić co robią poszczególne elementy opisane powyżej. 

#### `Stride` - o co w nim chodzi?

W przypadku CNN, stride odpowiadał za to jak bardzo będziemy przesuwać się jądrem po danych (jak duży krok wykonujemy). W przypadku transponowanych konwolucji jest innaczej. Stride określa wypełnienie obrazka jakimiś wartościami (przeważnie 0). To znaczy załóżmy, że mamy pewną macierz 3x3 i zastosujmy na nią różne wartości stride.

$$
\begin{array}{cccc}
\textbf{Wejściowa macierz}&
\textbf{Stride = 1} &
\textbf{Stride = 2} &
\textbf{Stride = 3} \\
\begin{bmatrix}
1 & 2 & 3 \\
4 & 5 & 6 \\
7 & 8 & 9
\end{bmatrix}
&
\begin{bmatrix}
1 & 2 & 3 \\
4 & 5 & 6 \\
7 & 8 & 9
\end{bmatrix}
&
\begin{bmatrix}
1 & 0 & 2 & 0 & 3 \\
0 & 0 & 0 & 0 & 0 \\
4 & 0 & 5 & 0 & 6 \\
0 & 0 & 0 & 0 & 0 \\
7 & 0 & 8 & 0 & 9
\end{bmatrix}
&
\begin{bmatrix}
1 & 0 & 0 & 2 & 0 & 0 & 3 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 \\
4 & 0 & 0 & 5 & 0 & 0 & 6 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 \\
7 & 0 & 0 & 8 & 0 & 0 & 9
\end{bmatrix}
\end{array}
$$

Jak widać, dla danej wartości `stride` = S, wstawiamy S - 1 zer pomiędzy każdą kolumnę i wiersz. Stride jest więc bezpośrednio odpowiedzialny za "rozciąganie" wejściowego obrazka.

Warto odrazu zauważyć, że zastosowanie stride zwiększa rozmiar obrazka. Załóżmy, że mamy rozmiar `H_in x W_in`. Wówczas zastosowanie stride zwraca obrazek o wymiarach `S(H_in - 1) + 1 x S(W_in - 1) + 1`. Istotnie, wystarczy zliczyć ile zer dodajemy w każdym wymaiarze. Jest to ważne i przyda nam się później jak będziemy chcieli obliczyć wyjściowy wymiar.

#### `Padding` - o co w nim chodzi?

W przypadku CNN, padding określał ile wartości chcemy dodać na brzegu wejściowych danych zanim zastosujemy jądro. Mogło by się wydawać, że tak samo będzie w przypadku transponowanych konwolucji, ale tak nie jest. W tym przypadku, `padding` jest stosowany dopiero na sam koniec i będzie określał ile wartości chcemy uciąć z brzegu zmodyfikowanego (ostatecznego) obrazka. Idea polega na tym, że po zastosowaniu naszego jądra chcemy mieć dane odpowiedniej wymiarowości. Dokładniej mówiąc odejmujemy `P` pikseli wokół obrazka (z każdego wymiaru, to znaczy z każdej strony), gdzie `P` to własnie nasz padding.

Widać, że zastosowanie `paddingu` zmiejsza każdy wymiar o `2P` (bo jednym z "każdej strony").

#### `Kernel` - o co w nim chodzi?

Tutaj wiele się różni względem "zwykłych" CNN. `Kernel_size` określa wielkość jądra, którym będziemy działać na obrazek, ale <strong>po zastosowaniu `stride`</strong>. 
Idea jądra w transponowanych konwolucjach jest znacząco inna względem tego co było w CNN. Ideologicznie można o tym myśleć jak o działaniu, które z jednego piksela robi nowych `KxK`, ale przy zachowaniu tego co wspomnieliśmy już na początku, czyli nie wsadzamy byle jakich pikseli do obrazka, a chcemy takie, które najbardziej pasują, więc mnożymy ten piksel prze pewne wagi (czym właśnie jest nasze jądro). Spróbuje to teraz dokładniej opisać. Na początku bierzemy jeden piskel, a następnie bierzemy kernel, to znaczy w tym przypadku pewną siatkę K x K. Wartości tej siatki zostaną uzupełnione przez wartość naszego piksela pomnożonego przez odpwowiednie wagi - macierz K x K to macierz wag, którą mnożymy przez wartość liczbową piksela.

Wobec tego można zauważyć, że duże jądra będą powodowały silniejsze rozymycie informacji, natomiast małe jądra dadzą bardziej skoncentrowaną informację. Tak więc wszytsko zależy od tego jaki cel chcemy osiągnąć. 

<strong>Bardzo ważne jest to, że w przypadku transponowanych konwolucji nie specyfikujemy kroku jaki jądro będzie wykonywać</strong> - zawsze będzie się ono poruszało z krokiem 1! Należy również poruszyć jedną kwestię. Ze względnu na fakt, że jądro porusza się z krokiem 1 to niektóre piksele się pokryją w wyjściowym obrazku. Wówczas na na rozciągniętych pikselach, gdzie jądro na siebie najdzie będziemy sumować te wartości. Najlepiej widać to na diagramie

<img src="https://raw.githubusercontent.com/tomczj/ML24_25/main/Autoencoders/diagram.jpg" width="400">

Możemy teraz policzyć jak będzie wyglądał finalny wymiar przed zastosowaniem paddingu,a po `stride`.  Przyjmijmy, że mamy jądro rozmiaru `K x K`. Wówczas łatwo zauważyć, że do aktualnego wymiaru dodajemy `K-1`. Co za tym idzie po zastosowaniu stride i kernela mamy następujący wymiar obrazka 

`S(H_in - 1) + 1 + K - 1 x S(W_in - 1) + 1 + K - 1` = `S(H_in - 1) + K x S(W_in - 1) + K`


### Szybkie podsumowanie

Jak widać, stride, padding oraz kernel_size odpowiadają – odpowiednio – za rozciągnięcie, obcięcie i rozmycie danych (myślimy tu głównie o obrazkach, żeby zachować intuicję). Odpowiednie dobranie tych parametrów pozwala precyzyjnie kontrolować wymiar wyjściowego obrazka, który wiemy jak będzie wyglądał, ponieważ od poprzednich wartości wystarczy po prostu odjąć `2P`. Co za tym idzie końcowy wymiar obrazka będzie wynosił

`S(H_in - 1) + K - 2P x S(W_in - 1) + K - 2P`

## Jak zatem wygląda proces upsamplingu?

Udało się nam już opisać główną ideę. Teraz możemy opsisać algortytm krok po kroku. Tak jak wcześniej przyjmijmy 

- `H_in x W_in` - wysokość i szerokość naszego obrazka.
- `K x K` - rozmiar jądra jakim będziemy działać na obrazek. 
- `S` - stride. 
- `P` - padding.

<strong>Krok I </strong> - musimy sobie odpowiedzieć na pytanie jakie wymiary chcemy uzyskać.

Zaczynamy od ustalenia, jaki rozmiar chcemy uzyskać na wyjściu. W przypadku transponowanej konwolucji, rozmiar wynikowy H_out × W_out możemy wyznaczyć ze wzoru (pokazaliśmy go wcześniej)

`H_out = S × (H_in - 1) + K - 2P`
`W_out = S × (W_in - 1) + K - 2P`

Ten wzór pozwala nam dobrać odpowiednie parametry (kernel, stride, padding), aby otrzymać pożądany rozmiar wyjściowy.

<strong>Krok II </strong> - rozmieszczenie zer, czyli `stride`

Transponowana konwolucja działa odwrotnie niż zwykła – przed właściwym „splotem” obraz jest rozrzedzany. Między kolejne kolumny i wiersze wejściowej macierzy wstawiamy S - 1 zer - za to odpowiada właśnie stride. W następnym kroku na rozrzedzoną macierz zadziałamy jądrem.

<strong>Krok III </strong> - działanie jądrem

Używamy jądra K × K, aby z pojedynczego piksela wygenerować fragment o rozmiarze K × K. Oznacza to, że wartość tego piksela zostaje przemnożona przez macierz wag (czyli jądro), a wynik wstawiany do macierzy wynikowej (czyli fragmentu wynikowego obrazka). Następnie przechodzimy do kolejnego piksela w tymczasowej macierzy (z krokiem 1). Jeśli kilka takich fragmentów nakłada się na te same piksele, ich wartości się sumują — jak pokazano na poniższym schemacie.

<strong>Krok IV </strong> - padding

Na końcu dodajemy padding, czyli wyjściowy obrazek obcinamy - po P pikseli z każdego wymiaru.

Ogólnie otrzymujemy schemat (zobrazowany poniżej). W miejscach kropek w macierzy należy wstawić zera:

<img src="https://raw.githubusercontent.com/tomczj/ML24_25/main/Autoencoders/diagram2.jpg" width="600"/>



# Przykłady

Teraz omówimy proste przykłady dla różnych wartości paddingu i stride. 

### Przykład 1

<img src="https://raw.githubusercontent.com/tomczj/ML24_25/main/Autoencoders/exp2.jpg" width="600"/>

### Przykład 2

<img src="https://raw.githubusercontent.com/tomczj/ML24_25/main/Autoencoders/exp1.jpg" width="600"/>

